In [1]:
import torchtext
from torchtext.vocab import Vectors, GloVe
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from hw1.models import *
from hw1.evaluation import *

In [2]:
# Our input $x$
TEXT = torchtext.data.Field()
    
# Our labels $y$
LABEL = torchtext.data.Field(sequential=False)


train, val, test = torchtext.datasets.SST.splits(
    TEXT, LABEL,
    filter_pred=lambda ex: ex.label != 'neutral')

# Build vocab
TEXT.build_vocab(train)
LABEL.build_vocab(train)

train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits(
(train, val, test), batch_size=10, device=-1)

# Build the vocabulary with word embeddings
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.vocab.load_vectors(vectors=Vectors('wiki.simple.vec', url=url))

In [4]:
model = CBOW(TEXT, LABEL)
(cnt_correct, cnt_total) = model_eval(model, test)
print('Pre-training: %d of %d correct' % (cnt_correct, cnt_total))
# print(model.embeddings.weight[:10,:10])
trainer = TextTrainer(TEXT, LABEL, model)
trainer.train(train_iter, num_iter=100, skip_iter=10)
(cnt_correct, cnt_total) = model_eval(model, test)
print('Post-training: %d of %d correct' % (cnt_correct, cnt_total))
# print(model.embeddings.weight[:10,:10])

Pre-training: 748 of 1821 correct
Iteration 0, loss: 1.064087
Iteration 10, loss: 15.669604
Iteration 20, loss: 1.454635
Iteration 30, loss: 1.800614
Iteration 40, loss: 1.811376
Iteration 50, loss: 0.794657
Iteration 60, loss: 0.920348
Iteration 70, loss: 0.765425
Iteration 80, loss: 2.725664
Iteration 90, loss: 1.217239
Post-training: 1005 of 1821 correct


In [16]:
test_iter = torchtext.data.BucketIterator(train, train=False, batch_size=11,
                                              device=-1)
cnt = 0
for batch in test_iter:
    cnt += (batch.label == 0).sum().data[0]
print(cnt)

0
